In [1]:
import robotic as ry
import numpy as np
import time
from math import sqrt, atan, pi

C = ry.Config()
C.addFile("cross_map.g")

r1 = "r1"
r2 = "r2"
m1 = "m1"
m2 = "m2"
g1 = "g1"
g2 = "g2"

robots = [r1, r2]
m_objects = [m1, m2]
goals = [g1, g2]

def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=20, kOrder=2, enableCollisions=True)  
    komo.updateRootObjects(C)
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)


def collisions(config, path):
    
    intial_state = C.getFrameState()
    
    res = []
    
    counter = 0
    
    
    for state in path:
        if counter == 20:
            config.attach(r1, m1)
            config.attach(r2, m2)
        config.setJointState(state)
        config.computeCollisions()
        y, j = config.eval(ry.FS.accumulatedCollisions)
        if y > 0:
            res.append(counter)
        counter += 1
    
  
        
    C.setFrameState(intial_state)
    C.getFrame(m1).unLink()
    C.getFrame(m2).unLink()
    
    return res
        

In [2]:
def distance(state1, state2):
    return sqrt((abs(state1[0] - state2[0]) ** 2 + abs(state1[1] - state2[1]) ** 2))
    

def determine_wall(path_both, collision_states, second_robot_state):


    start_state = path_both[collision_states[0]][:3]
    finish_state = path_both[collision_states[-1]][:3]
    
    
    len_wall = distance(start_state, finish_state) * 3
        
    min_dist_state = start_state
    min_distance = float("inf")
    
    for i in collision_states:
        curr_dist = distance(path_both[i][:3], second_robot_state)
        if  curr_dist < min_distance:
            min_distance = curr_dist
            min_dist_state = path_both[i][:3]
    
    
    min_dist_state[0] -= 0.3
    min_dist_state[1] += 0.3
    
    reverse_slope = -1 * (abs(second_robot_state[0] - min_dist_state[0]) / abs(second_robot_state[1] - min_dist_state[1]))
    
    rotation_in_radian = pi/2 - atan(reverse_slope)
    
    min_dist_state[2] = -(rotation_in_radian)
    
    return (min_dist_state, len_wall)        
        

    
    
    
    
    
    
    
    

In [ ]:
# try to solve situation for both case 
komo_both, ret_both = komo_solver(robots, m_objects, goals)
print(f"Trying to solve for both objects\neq: {ret_both.eq}\nSos: {ret_both.sos}\nisFeasible: {ret_both.feasible}\n\n\n\n")


if ret_both.feasible:
    komo_both.view_play()

else:
    path = komo_both.getPath()
    
    collision_states = collisions(C, path)
        
    wall_state, wall_len = determine_wall(path, collision_states, C.getFrame(r2).getJointState())
    
    C.addFrame("wall", "world").setShape(ry.ST.ssBox, [0.001, wall_len, 0.7, 0]).setContact(1).setJoint(ry.JT.transXYPhi).setJointState(wall_state)
    C.getFrame("wall").unLink()
    
    C.view()
    
    komo_solo, ret_solo = komo_solver(r2, m2, g2)
    
    komo_solo.view_play()
    
    print(ret_solo.feasible, ret_solo.eq, ret_solo.sos)
    
    



Trying to solve for both objects
eq: 2.598975159189436
Sos: 1607.3076800259737
isFeasible: False




True 0.055222915847488736 809.7513290364739
